EUCTRegister Scaper

In [5]:
from requests_html import HTMLSession
import pandas as pd
import warnings
import os 

warnings.filterwarnings('ignore')
%run "C:\Users\Diogo\OneDrive - NOVAIMS\My Functions\TG_message.py"

url = "https://www.clinicaltrialsregister.eu/"
s = HTMLSession()
page = s.get(url+ 'ctr-search/search?query=&page=')
last_page_num = max([int(a.split('=')[-1]) for a in page.html.links if 'page' in a]) 
s.close()
segment_size = 40

'''
Trough trial and error we found that the site was ending the connection after roughly 50 trials had been scraped
so we decided to do it in batches, restarting the connection each batch.
'''

try:
    for segment_start in range(0, last_page_num, segment_size):
        # create a new session for each segment
        s = HTMLSession()

        df = pd.DataFrame()
        segment_end = min(segment_start + segment_size, last_page_num)

        for page_num in range(segment_start, segment_end):
            page = s.get(url+ f'ctr-search/search?query=&page={page_num}')
            studies_links = [link for link in page.html.links if link.startswith('/ctr-search/trial') and 'results' not in link]

            for study_link in studies_links:
                study = s.get(url+ study_link)
                #Study Summary
                sum_field = study.html.find('td.cellGrey')                                      #All summary fields
                sum_field_value = study.html.find('td.cellLighterGrey')                         #All summary values
                study_info= {x.text:y.text for x,y in zip(sum_field,sum_field_value)} 

                #Study Details
                details = study.html.find('tr.tricell')                                         # List of all the other field and values (objects)
                for detail in details:
                    detail_list = detail.text.split('\n')                                       # Get text of all objects and split to get a list for each filed,value
                    if len(detail_list) < 3:
                        detail_list.append('')                                                  # Add blanks for fields without values 
                    study_info[detail_list[1]] = ' '. join(detail_list[2:])                   # Creates dict with Filed and Value(some fields have multiple values, must clean later if needed)

                df = df.append(study_info, ignore_index=True)

            # save the DataFrame every 40 pages
            if page_num % segment_size == 0 or page_num == segment_end - 1:
                file_name = fr"C:\Users\Diogo\OneDrive - NOVAIMS\PG - EDSA\3.PfDS - Programing for Data Science\Group Project\Group Project\Rep\Programming-Project\EU_data\EUCTRegister_{segment_start}-{page_num}.csv"
                df.to_csv(file_name, index=False)
                df = pd.DataFrame()  # reset the DataFrame to empty

        s.close()  # close the session after each segment
    message('Success!')

except:
    message('ERROR')


In [11]:
#Merging all files into a single csv

folder = r"C:\Users\Diogo\OneDrive - NOVAIMS\PG - EDSA\3.PfDS - Programing for Data Science\Group Project\Group Project\Rep\Programming-Project\EU_data"
df_list = []

for filename in os.listdir(folder):
    filepath = os.path.join(folder, filename)
    df = pd.read_csv(filepath, low_memory=False)
    df_list.append(df)


df = pd.concat(df_list, ignore_index=True)

df.to_csv('EuCTRegister.csv', index=False)